#imports

In [12]:
#installs
!pip install torch_summary
from torchsummary import summary

In [13]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil


#torch
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from torchvision import transforms

import cv2
from PIL import Image

#plot
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


#getting the dataset

In [3]:
#installing kaggle to access the dataset
!pip install kaggle

In [ ]:
#upload the api .json file
from google.colab import files
files.upload()

In [5]:
#move the file to the correct directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#download the dataset
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.18G/5.20G [00:52<00:00, 180MB/s]
100% 5.20G/5.20G [00:52<00:00, 107MB/s]


In [7]:
#unzip the dataset
!unzip skin-cancer-mnist-ham10000.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: /content/ham10000_images_part_2/ISIC_002

#preview dataset

In [8]:
#preview the dataset contents
!ls

ham10000_images_part_1	HAM10000_metadata.csv  hmnist_8_8_RGB.csv
HAM10000_images_part_1	hmnist_28_28_L.csv     kaggle.json
ham10000_images_part_2	hmnist_28_28_RGB.csv   sample_data
HAM10000_images_part_2	hmnist_8_8_L.csv       skin-cancer-mnist-ham10000.zip


In [9]:
#the non-img features file
pd.read_csv('HAM10000_metadata.csv').head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [10]:
# Define the function to create the test dataset
def create_test_dataset(source_dir, destination_dir, num_files):
    os.makedirs(destination_dir, exist_ok=True)

    all_files = os.listdir(source_dir)

    for file_name in all_files[:num_files]:
        full_file_name = os.path.join(source_dir, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, destination_dir)

In [ ]:
#creating a smaller sample dataset
create_test_dataset("HAM10000_images_part_1","test",500)

#model arch.

In [14]:
#clear cuda cashe
torch.cuda.empty_cache()

In [15]:
#to create our data and targets from the dataset
def create_data_and_targets(img_folders, df, target_col):
    images = []
    labels = []
    additional_features = []

    img_to_target = dict(zip(df['image_id'], df[target_col]))
    img_to_age = dict(zip(df['image_id'], df['age']))
    img_to_sex = dict(zip(df['image_id'], df['sex']))
    img_to_localization = dict(zip(df['image_id'], df['localization']))

    sex_mapping = {'male': 0, 'female': 1}  # Example encoding for sex
    localization_mapping = {loc: idx for idx, loc in enumerate(df['localization'].unique())}

    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
        transforms.ToTensor()           # Convert images to tensor
    ])

    for folder in img_folders:
        for filename in os.listdir(folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_id = filename.split('.')[0]
                if image_id in img_to_target:
                    img_path = os.path.join(folder, filename)
                    image = Image.open(img_path).convert('RGB')
                    image = transform(image)
                    label = img_to_target[image_id]

                    # Fetch additional features
                    age = img_to_age[image_id]
                    sex = sex_mapping[img_to_sex[image_id]]
                    localization = localization_mapping[img_to_localization[image_id]]

                    images.append(image)
                    labels.append(label)
                    additional_features.append([age, sex, localization])

    images_tensor = torch.stack(images)

    if isinstance(labels[0], (int, float)):
        targets = torch.tensor(labels).long()
    else:
        label_to_idx = {label: idx for idx, label in enumerate(df[target_col].unique())}
        targets = torch.tensor([label_to_idx[label] for label in labels])

    additional_features_tensor = torch.tensor(additional_features, dtype=torch.float)

    return images_tensor, targets, additional_features_tensor


#to shuffle and split the data
def split_data(images, targets, additional_features, val_ratio):
    # Ensure that the number of images, targets, and additional features are the same
    assert len(images) == len(targets) == len(additional_features), "The number of images, targets, and additional features must match."

    # Convert the images, targets, and additional features to a list of tuples for shuffling
    data = list(zip(images, targets, additional_features))

    train_data, val_data = train_test_split(data, test_size=val_ratio, random_state=42)

    tr_images, tr_targets, tr_additional_features = zip(*train_data)
    val_images, val_targets, val_additional_features = zip(*val_data)

    # Convert the lists back into tensors
    tr_images = torch.stack(tr_images)
    tr_targets = torch.tensor(tr_targets)
    val_images = torch.stack(val_images)
    val_targets = torch.tensor(val_targets)

    tr_additional_features = torch.stack(tr_additional_features)
    val_additional_features = torch.stack(val_additional_features)

    return tr_images, tr_targets, tr_additional_features, val_images, val_targets, val_additional_features



In [16]:
img_folders = ["test"] #only 200 img from the dataset to test
df = pd.read_csv("HAM10000_metadata.csv") #the csv that contains target
target_col = "dx" #the target col
val_ratio = 0.2

#get images tensor and target tensor
images, targets, nonimg_features = create_data_and_targets(img_folders, df, target_col)

#split the data
tr_images, tr_targets, tr_nonimg, val_images, val_targets, val_nonimg  = split_data(images, targets, nonimg_features, val_ratio)

In [17]:
#check input shape
print(tr_images.shape)
print(len(tr_images))

#check the shape of non_img features
print(tr_nonimg.shape)


torch.Size([400, 3, 224, 224])
400
torch.Size([400, 3])


In [18]:
class DatasetHAM(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float()/255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features  = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(256 * 28 * 28, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    # Define layers within the function
    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    # Forward pass
    x = fc1(combined)
    x = bn1(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc3(x)

    return x


#training
def train_batch(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data():
    train = DatasetHAM(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [19]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()
non_image_model = get_non_image_model(input_size=3)


#models summary
summary(model)
summary(non_image_model)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            1,792
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Dropout: 1-5                           --
├─Conv2d: 1-6                            73,856
├─BatchNorm2d: 1-7                       256
├─ReLU: 1-8                              --
├─MaxPool2d: 1-9                         --
├─Conv2d: 1-10                           295,168
├─BatchNorm2d: 1-11                      512
├─ReLU: 1-12                             --
├─MaxPool2d: 1-13                        --
├─Flatten: 1-14                          --
├─Linear: 1-15                           51,380,480
├─ReLU: 1-16                             --
├─BatchNorm1d: 1-17                      512
├─ReLU: 1-18                             --
├─Dropout: 1-19                          --
├─Linear: 1-20                           1,799
Total params: 51,754,503
Trainable params: 5

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            512
├─BatchNorm1d: 1-2                       256
├─ReLU: 1-3                              --
├─Dropout: 1-4                           --
├─Linear: 1-5                            8,256
├─BatchNorm1d: 1-6                       128
├─ReLU: 1-7                              --
├─Dropout: 1-8                           --
├─Linear: 1-9                            2,080
Total params: 11,232
Trainable params: 11,232
Non-trainable params: 0

In [ ]:
epox = 5
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

model.to(device)
non_image_model.to(device)

for epoch in range(epox):

    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss = train_batch(x, y, additional_features, model, non_image_model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(trn_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct = accuracy(x, y, model)
        validation_loss = val_loss(x, y, model)
    val_epoch_accuracy = np.mean(val_is_correct)

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_losses.append(validation_loss)
    val_accuracies.append(val_epoch_accuracy)

epochs = np.arange(epox) + 1


In [ ]:
# Create subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss", "Training and Validation Accuracy"))

# Plot training and validation loss
fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

# Plot training and validation accuracy
fig.add_trace(go.Scatter(x=epochs, y=train_accuracies, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=epochs, y=val_accuracies, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_xaxes(title_text='Epochs', row=2, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Accuracy', row=2, col=1)

# Format y-axis as percentage
fig.update_yaxes(tickformat=".0%", row=2, col=1)

# Add grid lines
fig.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with CNN")
fig.show()

#comparison

In [ ]:
#version 0

class DatasetHAM(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float()/255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features  = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(256 * 28 * 28, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    # Define layers within the function
    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    # Forward pass
    x = fc1(combined)
    x = bn1(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc3(x)

    return x


#training
def train_batch(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()


@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data():
    train = DatasetHAM(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()
non_image_model = get_non_image_model(input_size=3)


#models summary
summary(model)
summary(non_image_model)



epox = 5
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

model.to(device)
non_image_model.to(device)




for epoch in range(epox):

    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss = train_batch(x, y, additional_features, model, non_image_model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(trn_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct = accuracy(x, y, model)
        validation_loss = val_loss(x, y, model)
    val_epoch_accuracy = np.mean(val_is_correct)

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_losses.append(validation_loss)
    val_accuracies.append(val_epoch_accuracy)

epochs = np.arange(epox) + 1


# Create subplots
fig = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss", "Training and Validation Accuracy"))

# Plot training and validation loss
fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

# Plot training and validation accuracy
fig.add_trace(go.Scatter(x=epochs, y=train_accuracies, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=epochs, y=val_accuracies, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_xaxes(title_text='Epochs', row=2, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Accuracy', row=2, col=1)

# Format y-axis as percentage
fig.update_yaxes(tickformat=".0%", row=2, col=1)

# Add grid lines
fig.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with CNN")
fig.show()

In [ ]:
# Version 1: Using SGD optimizer with momentum

class DatasetHAM_V1(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float() / 255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model_v1():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(256 * 28 * 28, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=1e-4, momentum=0.9)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model_v1(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model_v1(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    x = fc1(combined)
    x = bn1(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc3(x)

    return x

#training
def train_batch_v1(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def accuracy_v1(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data_v1():
    train = DatasetHAM_V1(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM_V1(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss_v1(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

trn_dl_v1, val_dl_v1 = get_data_v1()
model_v1, loss_fn_v1, optimizer_v1 = get_model_v1()
non_image_model_v1 = get_non_image_model_v1(input_size=3)

summary(model_v1)
summary(non_image_model_v1)

epox_v1 = 5
train_losses_v1, train_accuracies_v1 = [], []
val_losses_v1, val_accuracies_v1 = [], []

model_v1.to(device)
non_image_model_v1.to(device)

for epoch in range(epox_v1):
    train_epoch_losses_v1, train_epoch_accuracies_v1 = [], []
    for ix, batch in enumerate(iter(trn_dl_v1)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss_v1 = train_batch_v1(x, y, additional_features, model_v1, non_image_model_v1, optimizer_v1, loss_fn_v1)
        train_epoch_losses_v1.append(batch_loss_v1)
    train_epoch_loss_v1 = np.array(train_epoch_losses_v1).mean()

    for ix, batch in enumerate(iter(trn_dl_v1)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct_v1 = accuracy_v1(x, y, model_v1)
        train_epoch_accuracies_v1.extend(is_correct_v1)
    train_epoch_accuracy_v1 = np.mean(train_epoch_accuracies_v1)

    for ix, batch in enumerate(iter(val_dl_v1)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct_v1 = accuracy_v1(x, y, model_v1)
        validation_loss_v1 = val_loss_v1(x, y, model_v1)
    val_epoch_accuracy_v1 = np.mean(val_is_correct_v1)

    train_losses_v1.append(train_epoch_loss_v1)
    train_accuracies_v1.append(train_epoch_accuracy_v1)
    val_losses_v1.append(validation_loss_v1)
    val_accuracies_v1.append(val_epoch_accuracy_v1)

epochs_v1 = np.arange(epox_v1) + 1

fig_v1 = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss - V1", "Training and Validation Accuracy - V1"))

fig_v1.add_trace(go.Scatter(x=epochs_v1, y=train_losses_v1, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig_v1.add_trace(go.Scatter(x=epochs_v1, y=val_losses_v1, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

fig_v1.add_trace(go.Scatter(x=epochs_v1, y=train_accuracies_v1, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig_v1.add_trace(go.Scatter(x=epochs_v1, y=val_accuracies_v1, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig_v1.update_xaxes(title_text='Epochs', row=1, col=1)
fig_v1.update_xaxes(title_text='Epochs', row=2, col=1)
fig_v1.update_yaxes(title_text='Loss', row=1, col=1)
fig_v1.update_yaxes(title_text='Accuracy', row=2, col=1)

fig_v1.update_yaxes(tickformat=".0%", row=2, col=1)

fig_v1.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with CNN - V1")
fig_v1.show()


In [ ]:
# Version 2: Adding extra convolutional layers

class DatasetHAM_V2(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float() / 255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model_v2():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(512, 1024, kernel_size=3, padding=1),
        nn.BatchNorm2d(1024),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(1024 * 7 * 7, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model_v2(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model_v2(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    x = fc1(combined)
    x = bn1(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc3(x)

    return x

#training
def train_batch_v2(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def accuracy_v2(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data_v2():
    train = DatasetHAM_V2(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM_V2(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss_v2(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

trn_dl_v2, val_dl_v2 = get_data_v2()
model_v2, loss_fn_v2, optimizer_v2 = get_model_v2()
non_image_model_v2 = get_non_image_model_v2(input_size=3)

summary(model_v2)
summary(non_image_model_v2)

epox_v2 = 5
train_losses_v2, train_accuracies_v2 = [], []
val_losses_v2, val_accuracies_v2 = [], []

model_v2.to(device)
non_image_model_v2.to(device)

for epoch in range(epox_v2):
    train_epoch_losses_v2, train_epoch_accuracies_v2 = [], []
    for ix, batch in enumerate(iter(trn_dl_v2)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss_v2 = train_batch_v2(x, y, additional_features, model_v2, non_image_model_v2, optimizer_v2, loss_fn_v2)
        train_epoch_losses_v2.append(batch_loss_v2)
    train_epoch_loss_v2 = np.array(train_epoch_losses_v2).mean()

    for ix, batch in enumerate(iter(trn_dl_v2)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct_v2 = accuracy_v2(x, y, model_v2)
        train_epoch_accuracies_v2.extend(is_correct_v2)
    train_epoch_accuracy_v2 = np.mean(train_epoch_accuracies_v2)

    for ix, batch in enumerate(iter(val_dl_v2)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct_v2 = accuracy_v2(x, y, model_v2)
        validation_loss_v2 = val_loss_v2(x, y, model_v2)
    val_epoch_accuracy_v2 = np.mean(val_is_correct_v2)

    train_losses_v2.append(train_epoch_loss_v2)
    train_accuracies_v2.append(train_epoch_accuracy_v2)
    val_losses_v2.append(validation_loss_v2)
    val_accuracies_v2.append(val_epoch_accuracy_v2)

epochs_v2 = np.arange(epox_v2) + 1

fig_v2 = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss - V2", "Training and Validation Accuracy - V2"))

fig_v2.add_trace(go.Scatter(x=epochs_v2, y=train_losses_v2, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig_v2.add_trace(go.Scatter(x=epochs_v2, y=val_losses_v2, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

fig_v2.add_trace(go.Scatter(x=epochs_v2, y=train_accuracies_v2, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig_v2.add_trace(go.Scatter(x=epochs_v2, y=val_accuracies_v2, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig_v2.update_xaxes(title_text='Epochs', row=1, col=1)
fig_v2.update_xaxes(title_text='Epochs', row=2, col=1)
fig_v2.update_yaxes(title_text='Loss', row=1, col=1)
fig_v2.update_yaxes(title_text='Accuracy', row=2, col=1)

fig_v2.update_yaxes(tickformat=".0%", row=2, col=1)

fig_v2.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with CNN - V2")
fig_v2.show()


In [ ]:
# Version 3: Using pre-trained ResNet model for the image part

from torchvision import models

class DatasetHAM_V3(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float() / 255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model_v3():
    resnet = models.resnet18(pretrained=True)
    for param in resnet.parameters():
        param.requires_grad = False

    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, 256)
    resnet = resnet.to(device)

    model = nn.Sequential(
        resnet,
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model_v3(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model_v3(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    x = fc1(combined)
    x = bn1(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.ReLU()(x)
    x = dropout(x)
    x = fc3(x)

    return x

#training
def train_batch_v3(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def accuracy_v3(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data_v3():
    train = DatasetHAM_V3(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM_V3(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss_v3(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

trn_dl_v3, val_dl_v3 = get_data_v3()
model_v3, loss_fn_v3, optimizer_v3 = get_model_v3()
non_image_model_v3 = get_non_image_model_v3(input_size=3)

summary(model_v3)
summary(non_image_model_v3)

epox_v3 = 5
train_losses_v3, train_accuracies_v3 = [], []
val_losses_v3, val_accuracies_v3 = [], []

model_v3.to(device)
non_image_model_v3.to(device)

for epoch in range(epox_v3):
    train_epoch_losses_v3, train_epoch_accuracies_v3 = [], []
    for ix, batch in enumerate(iter(trn_dl_v3)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss_v3 = train_batch_v3(x, y, additional_features, model_v3, non_image_model_v3, optimizer_v3, loss_fn_v3)
        train_epoch_losses_v3.append(batch_loss_v3)
    train_epoch_loss_v3 = np.array(train_epoch_losses_v3).mean()

    for ix, batch in enumerate(iter(trn_dl_v3)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct_v3 = accuracy_v3(x, y, model_v3)
        train_epoch_accuracies_v3.extend(is_correct_v3)
    train_epoch_accuracy_v3 = np.mean(train_epoch_accuracies_v3)

    for ix, batch in enumerate(iter(val_dl_v3)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct_v3 = accuracy_v3(x, y, model_v3)
        validation_loss_v3 = val_loss_v3(x, y, model_v3)
    val_epoch_accuracy_v3 = np.mean(val_is_correct_v3)

    train_losses_v3.append(train_epoch_loss_v3)
    train_accuracies_v3.append(train_epoch_accuracy_v3)
    val_losses_v3.append(validation_loss_v3)
    val_accuracies_v3.append(val_epoch_accuracy_v3)

epochs_v3 = np.arange(epox_v3) + 1

fig_v3 = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss - V3", "Training and Validation Accuracy - V3"))

fig_v3.add_trace(go.Scatter(x=epochs_v3, y=train_losses_v3, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig_v3.add_trace(go.Scatter(x=epochs_v3, y=val_losses_v3, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

fig_v3.add_trace(go.Scatter(x=epochs_v3, y=train_accuracies_v3, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig_v3.add_trace(go.Scatter(x=epochs_v3, y=val_accuracies_v3, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig_v3.update_xaxes(title_text='Epochs', row=1, col=1)
fig_v3.update_xaxes(title_text='Epochs', row=2, col=1)
fig_v3.update_yaxes(title_text='Loss', row=1, col=1)
fig_v3.update_yaxes(title_text='Accuracy', row=2, col=1)

fig_v3.update_yaxes(tickformat=".0%", row=2, col=1)

fig_v3.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with ResNet - V3")
fig_v3.show()


In [ ]:
# Version 4: Using LeakyReLU activation functions

class DatasetHAM_V4(Dataset):
    def __init__(self, x, y, additional_features):
        x = x.float() / 255
        x = x.view(-1, 3, 224, 224)
        self.x, self.y = x, y
        self.additional_features = additional_features

    def __getitem__(self, ix):
        x, y, additional_features = self.x[ix], self.y[ix], self.additional_features[ix]
        return x.to(device), y.to(device), additional_features.to(device)

    def __len__(self):
        return len(self.x)

#img model
def get_model_v4():
    model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(negative_slope=0.01),
        nn.MaxPool2d(2),
        nn.Dropout(0.25),

        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(negative_slope=0.01),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(negative_slope=0.01),
        nn.MaxPool2d(2),

        nn.Conv2d(256, 512, kernel_size=3, padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(negative_slope=0.01),
        nn.MaxPool2d(2),

        nn.Conv2d(512, 1024, kernel_size=3, padding=1),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(negative_slope=0.01),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(1024 * 7 * 7, 256),
        nn.LeakyReLU(negative_slope=0.01),
        nn.BatchNorm1d(256),
        nn.LeakyReLU(negative_slope=0.01),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    return model, loss_fn, optimizer

#non-img model
def get_non_image_model_v4(input_size):
    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.BatchNorm1d(128),
        nn.LeakyReLU(negative_slope=0.01),
        nn.Dropout(0.5),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(negative_slope=0.01),
        nn.Dropout(0.5),
        nn.Linear(64, 32)
    ).to(device)
    return model

#fusion model
def fusion_model_v4(image_output, non_image_output):
    combined = torch.cat((image_output, non_image_output), dim=1)

    fc1 = nn.Linear(combined.size(1), 256)
    bn1 = nn.BatchNorm1d(256)
    fc2 = nn.Linear(256, 128)
    bn2 = nn.BatchNorm1d(128)
    dropout = nn.Dropout(0.5)
    fc3 = nn.Linear(128, 7)

    x = fc1(combined)
    x = bn1(x)
    x = nn.LeakyReLU(negative_slope=0.01)(x)
    x = dropout(x)
    x = fc2(x)
    x = bn2(x)
    x = nn.LeakyReLU(negative_slope=0.01)(x)
    x = dropout(x)
    x = fc3(x)

    return x

#training
def train_batch_v4(x, y, additional_features, model, non_image_model, optimizer, loss_fn):
    model.train()
    non_image_model.train()

    x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
    model = model.to(device)
    non_image_model = non_image_model.to(device)

    optimizer.zero_grad()
    outputs = model(x)
    non_image_outputs = non_image_model(additional_features)
    combined_outputs = torch.cat((outputs, non_image_outputs), dim=1)

    loss = loss_fn(combined_outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def accuracy_v4(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

def get_data_v4():
    train = DatasetHAM_V4(tr_images, tr_targets, tr_nonimg)
    trn_dl = DataLoader(train, batch_size=64, shuffle=True)
    val = DatasetHAM_V4(val_images, val_targets, val_nonimg)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss_v4(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

trn_dl_v4, val_dl_v4 = get_data_v4()
model_v4, loss_fn_v4, optimizer_v4 = get_model_v4()
non_image_model_v4 = get_non_image_model_v4(input_size=3)

summary(model_v4)
summary(non_image_model_v4)

epox_v4 = 5
train_losses_v4, train_accuracies_v4 = [], []
val_losses_v4, val_accuracies_v4 = [], []

model_v4.to(device)
non_image_model_v4.to(device)

for epoch in range(epox_v4):
    train_epoch_losses_v4, train_epoch_accuracies_v4 = [], []
    for ix, batch in enumerate(iter(trn_dl_v4)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        batch_loss_v4 = train_batch_v4(x, y, additional_features, model_v4, non_image_model_v4, optimizer_v4, loss_fn_v4)
        train_epoch_losses_v4.append(batch_loss_v4)
    train_epoch_loss_v4 = np.array(train_epoch_losses_v4).mean()

    for ix, batch in enumerate(iter(trn_dl_v4)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        is_correct_v4 = accuracy_v4(x, y, model_v4)
        train_epoch_accuracies_v4.extend(is_correct_v4)
    train_epoch_accuracy_v4 = np.mean(train_epoch_accuracies_v4)

    for ix, batch in enumerate(iter(val_dl_v4)):
        x, y, additional_features = batch
        x, y, additional_features = x.to(device), y.to(device), additional_features.to(device)
        val_is_correct_v4 = accuracy_v4(x, y, model_v4)
        validation_loss_v4 = val_loss_v4(x, y, model_v4)
    val_epoch_accuracy_v4 = np.mean(val_is_correct_v4)

    train_losses_v4.append(train_epoch_loss_v4)
    train_accuracies_v4.append(train_epoch_accuracy_v4)
    val_losses_v4.append(validation_loss_v4)
    val_accuracies_v4.append(val_epoch_accuracy_v4)

epochs_v4 = np.arange(epox_v4) + 1

fig_v4 = make_subplots(rows=2, cols=1, subplot_titles=("Training and Validation Loss - V4", "Training and Validation Accuracy - V4"))

fig_v4.add_trace(go.Scatter(x=epochs_v4, y=train_losses_v4, mode='lines+markers', name='Training loss', line=dict(color='blue')), row=1, col=1)
fig_v4.add_trace(go.Scatter(x=epochs_v4, y=val_losses_v4, mode='lines+markers', name='Validation loss', line=dict(color='red')), row=1, col=1)

fig_v4.add_trace(go.Scatter(x=epochs_v4, y=train_accuracies_v4, mode='lines+markers', name='Training accuracy', line=dict(color='blue')), row=2, col=1)
fig_v4.add_trace(go.Scatter(x=epochs_v4, y=val_accuracies_v4, mode='lines+markers', name='Validation accuracy', line=dict(color='red')), row=2, col=1)

fig_v4.update_xaxes(title_text='Epochs', row=1, col=1)
fig_v4.update_xaxes(title_text='Epochs', row=2, col=1)
fig_v4.update_yaxes(title_text='Loss', row=1, col=1)
fig_v4.update_yaxes(title_text='Accuracy', row=2, col=1)

fig_v4.update_yaxes(tickformat=".0%", row=2, col=1)

fig_v4.update_layout(showlegend=True, grid=dict(rows=2, columns=1, pattern='independent'), title="Training and Validation Metrics with LeakyReLU - V4")
fig_v4.show()
